In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/evs-one-electric-vehicle-dataset/ElectricCarData_Clean.csv
/kaggle/input/evs-one-electric-vehicle-dataset/ElectricCarData_Norm.csv


In [2]:
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn import set_config

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [3]:
data=pd.read_csv("/kaggle/input/evs-one-electric-vehicle-dataset/ElectricCarData_Norm.csv",encoding='utf-8')

***EDA***

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Brand        103 non-null    object
 1   Model        103 non-null    object
 2   Accel        103 non-null    object
 3   TopSpeed     103 non-null    object
 4   Range        103 non-null    object
 5   Efficiency   103 non-null    object
 6   FastCharge   103 non-null    object
 7   RapidCharge  103 non-null    object
 8   PowerTrain   103 non-null    object
 9   PlugType     103 non-null    object
 10  BodyStyle    103 non-null    object
 11  Segment      103 non-null    object
 12  Seats        103 non-null    int64 
 13  PriceEuro    103 non-null    int64 
dtypes: int64(2), object(12)
memory usage: 11.4+ KB


In [5]:
data.head()

,Brand,Model,Accel,TopSpeed,Range,Efficiency,FastCharge,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,Tesla,Model 3 Long Range Dual Motor,4.6 sec,233 km/h,450 km,161 Wh/km,940 km/h,Rapid charging possible,All Wheel Drive,Type 2 CCS,Sedan,D,5,55480
1,Volkswagen,ID.3 Pure,10.0 sec,160 km/h,270 km,167 Wh/km,250 km/h,Rapid charging possible,Rear Wheel Drive,Type 2 CCS,Hatchback,C,5,30000
2,Polestar,2,4.7 sec,210 km/h,400 km,181 Wh/km,620 km/h,Rapid charging possible,All Wheel Drive,Type 2 CCS,Liftback,D,5,56440
3,BMW,iX3,6.8 sec,180 km/h,360 km,206 Wh/km,560 km/h,Rapid charging possible,Rear Wheel Drive,Type 2 CCS,SUV,D,5,68040
4,Honda,e,9.5 sec,145 km/h,170 km,168 Wh/km,190 km/h,Rapid charging possible,Rear Wheel Drive,Type 2 CCS,Hatchback,B,4,32997


In [6]:
data.isnull().sum()

Brand          0
Model          0
Accel          0
TopSpeed       0
Range          0
Efficiency     0
FastCharge     0
RapidCharge    0
PowerTrain     0
PlugType       0
BodyStyle      0
Segment        0
Seats          0
PriceEuro      0
dtype: int64

**Str Handling**

In [7]:
# Handling Top speed type
data['TopSpeed (km/h)'] = data['TopSpeed'].apply(lambda x: x.split()[0]).astype('float64')
# Handling Acceleration
data['Accel (sec)'] = data['Accel'].apply(lambda x: x.split()[0]).astype('float64')
# Handling Range
data['Range (km)'] = data['Range'].apply(lambda x: x.split()[0]).astype('float64')
# Handling Power Efficiency
data['Efficiency (Wh/km)'] = data['Efficiency'].apply(lambda x: x.split()[0]).astype('float64')
# Handling Fast Charge
data['FastCharge'] = data['FastCharge'].apply(lambda x: x.replace('-',str(0)))
data['FastCharge (km/h)'] = data['FastCharge'].apply(lambda x: x.split()[0]).astype('float64')

**Categorical Data Handling**

In [8]:
# def oneHotEndcoder(transformer,data_former_numerical,data_former_category):
#   transformer.fit(data_former_category)
#   data_after_transform = pd.DataFrame(data=transformer.transform(data_former_category).toarray(),columns=transformer.get_feature_names_out())
#   return transformer,pd.concat([data_former_numerical,data_after_transform],axis=1)


In [9]:

# ohe_encoder = OneHotEncoder(sparse=True)
# cat_cols=list(data.drop(columns=['TopSpeed','Accel','Range','Efficiency','FastCharge']).select_dtypes(include=['object']).columns)



# data_former_category = data.loc[:,cat_cols]
# data_former_numerical = data.drop(columns = cat_cols)
# ohe_encoder, data = oneHotEndcoder(ohe_encoder,data_former_numerical,data_former_category)



In [10]:
# data.head()

****top10FastestEV****

In [11]:
top10FastestEV=data.sort_values(['TopSpeed'], ascending=False).reset_index().iloc[0:10]

In [12]:
top10FastestEV['CarName'] = top10FastestEV['Brand']+top10FastestEV['Model']

In [13]:
# f=plt.figure(figsize=(12,4))
# ax=f.add_subplot(111)
# sns.barplot(x=top10FastestEV['Brand']+top10FastestEV['Model'], y=top10FastestEV['TopSpeed'])
# ax.set_xticklabels(ax.get_xticklabels,rotation='45')


In [14]:
px.bar(top10FastestEV,x='CarName', y='TopSpeed (km/h)',hover_data=['Accel','PowerTrain'],color='TopSpeed',text_auto='.2s',title='Top 10 Fastest EVs')

**Speed distribution**

In [15]:
topspeed_cate = pd.cut(data['TopSpeed (km/h)'], bins=[0,50,100,150,200,250,300,350,400,450], labels=["0-50","50-100","100-150","150-200","200-250","250-300","300-350","350-400","400-450"]) 

In [16]:
# fig = ff.create_distplot([topspeed_cate], group_labels=['Top Speed Range'])

fig = px.histogram(topspeed_cate,text_auto=True)
fig.show()

In [17]:
type(data['TopSpeed (km/h)'])

pandas.core.series.Series